In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Second-order advection

To get a more accurate solution, we need to handle both the space and time discretizations in a higher order fashion.

First, let's look again at the definition of the average:

$$\langle a \rangle_i = \frac{1}{\Delta x} \int_{x_{i-1/2}}^{x_{i+1/2}} a(x) dx$$

let's substitute in a Taylor expansion about $x_i$:

$$a(x) = a(x_i) + \left . \frac{\partial a}{\partial x} \right |_i (x - x_i) + \frac{1}{2} \left . \frac{\partial^2 a}{\partial x^2} \right |_i (x - x_i)^2 + \mathcal{O}(\Delta x^3)$$

inserting this into our integral, we have:

$$\langle a \rangle_i = \frac{1}{\Delta x} \int_{x_{i-1/2}}^{x_{i+1/2}} 
    \left [a(x_i) + \left . \frac{\partial a}{\partial x} \right |_i (x - x_i) + \frac{1}{2} \left . \frac{\partial^2 a}{\partial x^2} \right |_i (x - x_i)^2 + \mathcal{O}(\Delta x^3) \right ] dx$$

Look at the terms:

* the first term is just a constant, so it's integral is just $a(x_i) \Delta x$
* the second term is odd over the interface $[x_{i-1/2},x_{i+1/2}]$, so it integrates to 0
* the third term will integrate to something like $(\Delta x)^3$ (before we divide by the $\Delta x$ out front)

This means that

$$\langle a \rangle_i = a(x_i) + \mathcal{O}(\Delta x^2)$$

So if we work to second-order in space, then we don't need to worry about the difference between the cell average and cell-center.  

To make the notation easier, we'll drop the $\langle \rangle$ from now forward.

There are 2 broad ways that we can handle the time integration to get more accuracy:

* Predict our interface states at the midpoint in time using a Taylor expansion (in both space and time).  This would result in a time-centered interface state.  This procedure is sometimes called *characteristic tracing*.  

* Just construct high-order interface states in space, ignoring how they evolve in time, and use a high-order integrator (like Runge-Kutta) to handle the time integration.  This is called the *method-of-lines* approach.

We'll focus on the second approach (method-of-lines).  For details on the first, see my notes: [Computational Hydrodynamics for Astrophysics](https://github.com/Open-Astrophysics-Bookshelf/numerical_exercises)